In [1]:
import pygame
import time
import random
import neat
import os
pygame.font.init()  # init font

pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initial inputs for our game 
X_screen = 500                                    # width of screen
Y_screen = 800                                    # length of screen
bird_image = pygame.image.load("bird.png")        # image of our object bird
super_image = pygame.image.load("super.png")        # image of our object bird
super_image = pygame.transform.scale(super_image, (bird_image.get_width()/2, bird_image.get_height()/2)) #chganging size of picture
STAT_FONT = pygame.font.SysFont("comicsans", 50)  # font of score drawn on screen

In [3]:
# Initial position of bird
bird_X = X_screen * 0.15   # initial x position of bird which is constant during game
bird_Y = Y_screen * 0.4    # initial y position of bird

In [4]:
# Assumptions for obstacles
rectangle_width = 70      # width of our obstacle
rectangle_gap = 150       # gap between top and bottom obstacle

In [5]:
class Bird:
    """
    Class Bird is representation of our bird in game. 
    It has functions up() which is used to move bird up. 
    Function change_position changing position of bird up or down, depends on player input. 
    Next function draw() draw bird on display and also maintain limitation of position of our bird.
    """
    
    def __init__(self, bird_X, bird_Y):
        self.x = bird_X          # x position of bird
        self.y = bird_Y          # y position of bird
        self.move = -7           # constant move of bird. It is negative because it is an upward move
        self.image = bird_image  # image of bird
        self.repeat = 0          # it shows how long program works without user action
        self.length = bird_image.get_height()
    
    def up(self):
        self.repeat = 0          # if user want to jump repeat is equal 0 
    
    
    def change_position(self):
        self.repeat += 1         # if the function is called 
        self.y = self.y + self.move*self.repeat + self.repeat**2  # this is line which is responsible for changing position of bird
                                                                  # if user don't call function up() the value becomes larger as the program runs
    
    def collision(self):                                        
        end = False
        if self.y < 0:          # checking if bird touch the top or bottom of screen, if so game ends
            end = True
        if self.y > (Y_screen - self.length):
            end = True
        return end
            
    
    def draw(self, screen):
        self.image = bird_image
        screen.blit(self.image, (self.x, self.y))                 # drawing bird based on new y position

In [6]:
class Obstacle:
    """
    Next class Obstacle generate obstacles in our game. There are four functions:
    First one set_high() generate random height of obstacle in range between 200 and 450. 
    The second function: change_position() is responsible for move of pipes. 
    it also checking if obstacle is out of window. If yes it creating new obstacle with new random height.
    Next function draw() creating a rectangles which are displayed on screen. 
    The last function collide() is checking is there a collision between bird and created obstacles. if yes it returns False. 
    """
    
    def __init__(self, x):
        self.x = x                       # x position of obstacle. By default size of screen (500)
        self.height = 0                  # height of top obstacle
        self.score = 0                   # how many obstacles player passed
        self.width = rectangle_width     # width of obstacle
        self.gap = rectangle_gap         # gap between top and bottom obstacle
        self.move = -3                   # speed of obstacle
        self.Y_screen = Y_screen         # size of screen 
        self.set_height()                # setting the initial value of height
        
    def set_height(self):
        self.height = random.randrange(200, 450)     # random value of height between 200 and 450 
    
    def change_position(self):                       
        self.x += self.move                          # changing position of obstacle based on initial value of speed
        if self.x <= -self.width:                    # if obstacle miss the screen creating new obstacle at the beggining
            self.x = 500
            self.set_height()
            self.score += 1
        return self.score
            
    def draw(self, screen):
        pygame.draw.rect(screen, (0, 0, 0), (self.x, 0, self.width, self.height))                        # drawing top obstacle
        height_second = self.Y_screen - self.height - self.gap       # height of the second obstacle based on length of screen, gap and height of first one
        pygame.draw.rect(screen, (0, 0, 0), (self.x, self.height + self.gap, self.width, height_second)) # drawing bottom obstacle
        
    def collide(self, bird):
        end = True
        if bird.x in range(self.x - self.width, self.x):
            if bird.y <= self.height or bird.y >= (self.height + self.gap - 50):   # checking if bird is on the same level as obstacles
                end = False
        return end

In [7]:
class SuperMario:
    """
    Class defines the behavior and characteristics of the Super Mario object, 
    including movement, position, and collision detection
    """
    
    def __init__(self):
        self.x = 500                             # initial x position for super mario 
        self.y = 200                             # initial y position for super mario
        self.speed = -3                          # initial speed 
        self.score = 0                           # how many marios we passed
        self.set_speed()                         # setting the initial
        self.image = super_image                 # image of super mario
        self.width = super_image.get_width()     # width of picture super mario
        self.height = super_image.get_height()   # length of picture super mario
        
    def set_height(self, obst):
        top_y = obst.height                      # length of top obstacle
        bottom_y = obst.height + obst.gap        # length of bottom obstacle
        if random.random()>= 0.5:                # we chose random if we want our super mario to "fly" above or under gap
            self.y = random.randrange(self.width, top_y)    # new y position of super mario
        else:
            self.y = random.randrange(bottom_y, Y_screen - self.width)    # new y position of super mario
    
    def set_speed(self):
        self.speed = random.randrange(-10, -5)   # random speed between -10 and -5 
    
    def change_position(self, obst):             
        self.x += self.speed                     # changing position based on new speed
        if self.x <= -self.width:                # if mario miis the screen, creating new one
            self.x = 500
            self.set_height(obst)
            self.set_speed()
            self.score += 1
        return self.score                        # return score which depends on obstacles we passed
            
    def draw(self, screen):
        screen.blit(self.image, (self.x, self.y))
        
    def collide(self, bird):                      
        bird_rect = bird.image.get_rect(topleft=(bird.x, bird.y))   # position of bird
        mario_rect = self.image.get_rect(topleft=(self.x, self.y))  # position of mario
        if bird_rect.colliderect(mario_rect):                       # checking if bird hits mario
            return True
        return False

In [8]:
def draw_screen(screen, birds, obst, score2, super_mario1, super_mario2):
    """
    Function used in the main fitness function. 
    It is used to draw all the necessary elements and refresh the screen.
    """
    
    screen.fill((255, 255, 255))      # filling background with white colour
    obst.draw(screen)                 # drawing obstacles
    super_mario1.draw(screen)         # drawing first super mario
    super_mario2.draw(screen)         # drawing second super mario
    
    for bird in birds:                # drawing all birds
        bird.draw(screen)
    
    score_label = STAT_FONT.render("Score: " + str(score2),1,(0,255,0))      # defining text of score
    screen.blit(score_label, (X_screen - score_label.get_width() - 15, 10))  # drawing score on board
    pygame.display.update()           # displaying screen
    
    

In [9]:
def end_game(birds):
    if len(birds) == 0:  # if all birds death, the loop is over
            return True

In [10]:
def fitness(genomes, config):
    """
    The fitness function updates the position of birds in the game based on their neural network output. 
    If a bird successfully passes through an obstacle, its genome's fitness is increased, 
    while a collision results in a decrease in fitness. 
    The function removes birds with low fitness scores and their corresponding neural networks and genomes. 
    This process is repeated until the game is over.
    """
    
    nets = []         # list for neural network nets for each birds
    ge = []           # list of genomes (birds)
    birds = []        # list of objects birds
    
    for x, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)  # creating neural network net bassed on particular genom and config that are fixed
        nets.append(net)                                    # append neural network net to list
        birds.append(Bird(bird_X, bird_Y))                  # append object of bird to list
        g.fitness = 0                                       # fix fitness value of every genom as 0 
        ge.append(g)                                        # append genom to ge list
    
    screen = pygame.display.set_mode((X_screen, Y_screen))  # initial pygame display 

    
    obst = Obstacle(X_screen)                               # creating obstacle object
    
    
    super_mario1 = SuperMario()                             # creating two super mario obstacles
    super_mario2 = SuperMario()
    
    clock = pygame.time.Clock()                             # creating object time.clock() which helps with game speed management
    
    score1 = 0                                              # variables needed during calculations
    score3 = 0
    score5 = 0

    start = True
    while start:
        
        clock.tick(50)                       # setting tick for 50 in order to slow down the game
        for event in pygame.event.get():     #checking if user click the quit button 
            if event.type == pygame.QUIT:
                start = False
                pygame.quit()
        
        for x, bird in enumerate(birds):
            ge[x].fitness += 0.01           # if one loop pass we add 0.01 to every bird that survived
            bird.change_position()          # changing position of all birds
            
            # using inputs such to calculate if jump or not
            # first input: distance between y position of bird and position of top obstacle
            # second input: distance between y position of bird and position of bottom obstacle
            # third and fourth: Euclidean distance between the position of the "bird" and "super_mario1/2" 
            output = nets[x].activate((abs(bird.y - obst.height), abs(bird.y - obst.height - obst.gap+50), 
                                      ((bird.x - super_mario1.x)**2 + (bird.y - super_mario1.y)**2)**(1/2),
                                      ((bird.x - super_mario2.x)**2 + (bird.y - super_mario2.y)**2)**(1/2),
                                      abs(bird.y - super_mario1.y), abs(bird.y - super_mario2.y)))
            
            # if output of calculation is higher than 0.5 bird should jump 
            if output[0] > 0.5:
                bird.up()
        
        
        
        
        
        
        # this part of score is responsible for moving obstacles and also for countin score
        # it also check if our bird passed the obstacle, if so the bird get 5 points
        score2 = obst.change_position()
        if score2 > score1:
            for x, bird in enumerate(birds):
                ge[x].fitness += 5
        score1 = score2
        
        
        # this part of score is responsible for moving super_mario1 and also for countin score
        # it also check if our bird passed the super_mario1, if so the bird get 0.3 points
        score4 = super_mario1.change_position(obst)
        if score4 > score3:
            for x, bird in enumerate(birds):
                ge[x].fitness += 0.3
        score3 = score4
        
        
        # this part of score is responsible for moving super_mario2 and also for countin score
        # it also check if our bird passed the super_mario2, if so the bird get 0.3 points
        score6 = super_mario2.change_position(obst)
        if score4 > score3:
            for x, bird in enumerate(birds):
                ge[x].fitness += 0.3
        score5 = score6
        
        
        # Checking if bird hit the top or bottom part of screen
        for x, bird in enumerate(birds):
            if bird.collision() == True:
                ge[x].fitness -= 2
                birds.pop(x)
                nets.pop(x)
                ge.pop(x)
                
        if end_game(birds): break                           # checking if all bird are death
        
                
        # check if bird hit obstacle
        for x, bird in enumerate(birds):
            if obst.collide(bird) == False:
                ge[x].fitness -= 1
                birds.pop(x)
                nets.pop(x)
                ge.pop(x)
        if end_game(birds): break                           # checking if all bird are death
        
        
        #check if bird hits super_mario1/2
        for x, bird in enumerate(birds):
            if super_mario1.collide(bird) == True or super_mario2.collide(bird) == True:
                ge[x].fitness -= 5
                birds.pop(x)
                nets.pop(x)
                ge.pop(x)
                
        if end_game(birds): break                           # checking if all bird are death
            
        
        draw_screen(screen, birds, obst, score2, super_mario1, super_mario2)  #drawing all neccesary elements

In [11]:
def run(config_file):
    """
    This function contains all the assumptions for our algorithm based on config_file 
    which is file that represents all inputs that are needed to run the algorithm. 
    """
    
    #Arguments in function providethe default settings for the genomes, 
    #reproduction, species, and stagnation of the NEAT algorithm.
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)
    
    
    #based on config we create our population (50 chromosoms)
    population = neat.Population(config)
    
    # add statistic 
    population.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    population.add_reporter(stats)
    
    #our winner after 50 iterations
    winner = population.run(fitness, 50)
    print(winner)

In [12]:
if __name__ == "__main__":
    
    # upload file with assesments to model
    __file__ = os.path.abspath("Untitled.py")
    local_dir = os.path.dirname(__file__)
    config_path = os.path.join(local_dir, 'Config_file.txt')
    run(config_path)  # running algorithm 


 ****** Running generation 0 ****** 

Population's average fitness: -2.00680 stdev: 1.13363
Best fitness: 0.50000 - size: (1, 6) - species 1 - id 29
Average adjusted fitness: 0.510
Mean genetic distance 0.985, standard deviation 0.394
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    50      0.5    0.510     0
Total extinctions: 0
Generation time: 2.582 sec

 ****** Running generation 1 ****** 

Population's average fitness: -1.92660 stdev: 1.45012
Best fitness: 0.53000 - size: (1, 6) - species 1 - id 10
Average adjusted fitness: 0.523
Mean genetic distance 1.207, standard deviation 0.404
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    50      0.5    0.523     0
Total extinctions: 0
Generation time: 2.508 sec (2.545 average)

 ****** Running generation 2 ****** 

Population's average fitness: -1.74980 stdev: 1